In [21]:
import glob
import os, re 
import numpy as np
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:1])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord"]


In [6]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() 
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) 
    sentence = re.sub(r'[" "]+', " ", sentence) 
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) 
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>' 
    return sentence

print(preprocess_sentence(raw_corpus[0]))

<start> now i ve heard there was a secret chord <end>


In [5]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

In [8]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 12000,
        filters = ' ',
        oov_token = "<unk>"
    )
    tokenizer.fit_on_texts(corpus)
    
    tensor = tokenizer.texts_to_sequences(corpus)
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...   43    3    0]
 ...
 [   5   22    9 ...   10 1013    3]
 [  37   15 9049 ...  877  647    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fd8ec3e4cd0>


In [9]:
print(tensor[:3])
tensor.shape

[[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0
     0]
 [   2   17 2639  873    4    8   11 6043    6  329    3    0    0    0
     0]
 [   2   36    7   37   15  164  282   28  299    4   47    7   43    3
     0]]


(175749, 15)

In [10]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    
    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [11]:
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

print(src_input[0])
print(tgt_input[0])
src_input.shape

[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0]
[  50    5   91  297   65   57    9  969 6042    3    0    0    0    0]


(175749, 14)

In [12]:
from sklearn.model_selection import train_test_split

# tensor를 train, test 데이터로 분리
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


In [13]:
vocab_size = tokenizer.num_words + 1  # 단어사전의 단어 개수 + 0:<pad>
embedding_size = 256
hidden_size = 1024

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_size))
model.add(tf.keras.layers.LSTM(hidden_size, return_sequences=True))
model.add(tf.keras.layers.LSTM(hidden_size, return_sequences=True))
model.add(tf.keras.layers.Dense(vocab_size))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 256)         3072256   
_________________________________________________________________
lstm (LSTM)                  (None, None, 1024)        5246976   
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 1024)        8392704   
_________________________________________________________________
dense (Dense)                (None, None, 12001)       12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [14]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10)

Epoch 1/10
4394/4394 [==============================] - 147s 33ms/step - loss: 3.2644
Epoch 2/10
4394/4394 [==============================] - 143s 33ms/step - loss: 2.7927
Epoch 3/10
4394/4394 [==============================] - 143s 33ms/step - loss: 2.4952
Epoch 4/10
4394/4394 [==============================] - 143s 33ms/step - loss: 2.2302
Epoch 5/10
4394/4394 [==============================] - 143s 33ms/step - loss: 2.0036
Epoch 6/10
4394/4394 [==============================] - 143s 33ms/step - loss: 1.8086
Epoch 7/10
4394/4394 [==============================] - 143s 33ms/step - loss: 1.6416
Epoch 8/10
4394/4394 [==============================] - 143s 33ms/step - loss: 1.5003
Epoch 9/10
4394/4394 [==============================] - 143s 33ms/step - loss: 1.3827
Epoch 10/10
4394/4394 [==============================] - 143s 33ms/step - loss: 1.2866


In [15]:
# 문장 생성 함수
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [16]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love the way you dance with anybody <end> '

In [17]:
generate_text(model, tokenizer, init_sentence="<start> i was", max_len=20)

'<start> i was afraid of life and you came in time <end> '

In [18]:
generate_text(model, tokenizer, init_sentence="<start> i hate", max_len=20)

'<start> i hate a goofy especially <end> '

In [19]:
generate_text(model, tokenizer, init_sentence="<start> you are", max_len=20)

'<start> you are the only one <end> '

In [20]:
generate_text(model, tokenizer, init_sentence="<start> happy", max_len=20)

'<start> happy people yeah , yeah yeah , yeah <end> '